## Query Athena using ODBC driver
This example shows how to create a basic SQL query against an existing table defined in Athena, which consists of data imported from a simple two column CSV file.

The query runs against a predefined database called `sampledb` and a table called `users`

Sensitive and other configuration data used by the notebook, such as credentials, is passed to the host Docker container using environment variables. Other data is defined in the notebook, such as the S3 bucket location where results data is written.

It is possible to use standard IAM authentication credentials or temporary credentials created using `sts.assumerole()`, depending on whether the `AWS_ROLE_ARN` defines a role to be assumed.

Results are output to a pandas dataframe.

In [ ]:
import boto3
import pyodbc
import os
import pandas as pd

def role_arn_to_session(**args):
    """
    Usage :
        session = role_arn_to_session(
            RoleArn='arn:aws:iam::012345678901:role/example-role',
            RoleSessionName='ExampleSessionName')
        client = session.client('sqs')
    """
    client = boto3.client('sts',
        aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
        aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'] 
    )
    
    response = client.assume_role(**args)
    aws_access_key_id=response['Credentials']['AccessKeyId']
    aws_secret_access_key=response['Credentials']['SecretAccessKey']
    aws_session_token=response['Credentials']['SessionToken']
    
    return { "aws_access_key_id" : aws_access_key_id , "aws_secret_access_key" : aws_secret_access_key, "aws_session_token" : aws_session_token }

aws_kms_key=os.environ['AWS_S3_KMS_KEY'] 
aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID']
aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'] 
aws_region = os.environ['AWS_DEFAULT_REGION']

if os.environ.get('AWS_ROLE_ARN') != None :
    # Get the MFA token from your device
    mfa_token_code = input()
    temp_creds = role_arn_to_session(    
        RoleArn=os.environ['AWS_ROLE_ARN'],
        RoleSessionName='ExampleSessionName',
        SerialNumber=os.environ['AWS_MFA_SERIAL_NUMBER'],
        TokenCode=mfa_token_code,
    #    DurationSeconds=28800 # 8 hours
    )
    DSNTemplate="Driver=Simba Athena ODBC Driver 64-bit;AwsRegion={};S3OutputLocation=s3://pedanticism-athena-output/output/;S3OutputEncOption=SSE_KMS;S3OutputEncKMSKey={};AuthenticationType=IAM Credentials;UID={};PWD={};SessionToken={}"
    DSN = DSNTemplate.format(aws_region,aws_kms_key,temp_creds["aws_access_key_id"], temp_creds["aws_secret_access_key"], temp_creds["aws_session_token"])

else:
    DSNTemplate="Driver=Simba Athena ODBC Driver 64-bit;AwsRegion={};S3OutputLocation=s3://pedanticism-athena-output/output/;S3OutputEncOption=SSE_KMS;S3OutputEncKMSKey={};AuthenticationType=IAM Credentials;UID={};PWD={}" # ";SessionToken={}"
    DSN = DSNTemplate.format(aws_region,aws_kms_key, aws_access_key_id, aws_secret_access_key) 


## Execute the query against Athena and write the results to a dataframe

In [ ]:
# Specifying the ODBC driver, server name, database, etc. directly
import pandas as pd
cnxn = pyodbc.connect(DSN)
query = 'SELECT * FROM "sampledb"."users" limit 10'
df = pd.read_sql_query(query, cnxn)

df